In [1]:
import glob
import pathlib

import pandas as pd
pd.get_option("display.max_columns")
import plotly.express as px

from btree_analysis.log_parser import parse_experiment

In [33]:
EXPERIMENT_PATH = "/Users/artur/Documents/broadcast-tree/experiments/results/*"
EXPERIMENT_IDENTIFIER = ["payload_size", "flood", "unchanged_counter", "source_id", "iteration"]

In [34]:
events = []
dirs = glob.glob(EXPERIMENT_PATH)
num_dirs = len(dirs)
for count, experiment in zip(range(0, num_dirs), dirs):
    print(f"Parsing experiment {experiment} ({count + 1}/{num_dirs})")
    _events, _, _ = parse_experiment(pathlib.Path(experiment))
    events += _events
    
df = pd.DataFrame(events)

Parsing experiment /Users/artur/Documents/broadcast-tree/experiments/results/2022-07-28_14-28-12 (1/1)


# Evaluation Metrics
- [x] Table: How many nodes have received the entire payload?
- [x] Plot: Percentage comparison of BTP vs. flood w.r.t. to received payloads
- [X] Plot: How many percent more or less energy does BTP need compared to flood
- [X] Plot: What is the overhead of the BTP protocol (i.e. payload energy vs. protocol energy)?
- [X] Plot: How are individual nodes strained?

# Absolute Values of Received Payloads

In [35]:
reception_df = df.groupby(EXPERIMENT_IDENTIFIER)["event"].apply(lambda x: (x == 'receive').sum()).reset_index(name='count')

In [36]:
reception_df[reception_df["count"] == 0]

,payload_size,flood,unchanged_counter,source_id,iteration,count


In [37]:
fig = px.box(
    reception_df,
    x="payload_size",
    y="count", 
    color="unchanged_counter",
    facet_col="flood",
    facet_row="source_id"
)
fig

# Relative Values of Received Payloads

In [7]:
def reception_ratio(x):
    flooding = x[x["flood"] == True]
    btp = x[x["flood"] == False]
    
    return float(btp["count"]) / flooding["count"]

count_df = reception_df.groupby(["payload_size", "source_id", "unchanged_counter", "iteration"]).apply(reception_ratio).reset_index(name='ratio')

In [8]:
fig = px.box(
    count_df,
    x="payload_size",
    y="ratio", 
    color="unchanged_counter",
    facet_col="source_id"
)
fig

# Prepare energy df

In [11]:
def energy_ratio(x):
    energy_payload_df = x[x["frame_type"] == "payload"]
    energy_protocol_df = x[x["frame_type"] == "protocol"]
    
    energy_payload = float(energy_payload_df["mWb_total"])
    
    if energy_protocol_df.empty:
        energy_protocol = 0
    else:
        energy_protocol = float(energy_protocol_df["mWb_total"])
    
    total_energy = energy_payload + energy_protocol
    
    payload_rel = energy_payload / total_energy
    protocol_rel = energy_protocol / total_energy
    
    x.loc[x["frame_type"] == "payload", "mWb_rel"] = payload_rel
    x.loc[x["frame_type"] == "protocol", "mWb_rel"] = protocol_rel
    
    return x
    
    

df["mW"] = 10**(df["tx_pwr"]/10)
df["mWb"] = df["mW"] * (df["data_len"] + 34)

df.loc[(df["event"] == "send") & (df["message"] == "Successfully forwarded payload.") | (df["message"] == "Successfully sent next chunk."), "frame_type"] = "payload"
df.loc[(df["event"] == "send") & ((df["message"] != "Successfully forwarded payload.") & (df["message"] != "Successfully sent next chunk.")), "frame_type"] = "protocol"

energy_df = df[df["event"] == "send"].groupby(EXPERIMENT_IDENTIFIER + ["frame_type"])["mWb"].sum().reset_index(name='mWb_total')
energy_df = energy_df.groupby(EXPERIMENT_IDENTIFIER).apply(energy_ratio)#.reset_index()


TypeError: cannot convert the series to <class 'float'>

# Comparison between flooding and BTP, energy wise

In [61]:
def energy_ratio(x):
    flooding = x[(x["flood"] == True)].sum()
    btp = x[(x["flood"] == False)].sum()
    
    return btp["mWb_total"] / flooding["mWb_total"]

energy_ratio_df = energy_df.groupby(["payload_size", "source_id", "unchanged_counter", "iteration"]).apply(energy_ratio).reset_index(name='ratio')

/var/folders/26/2pp7vp553ms3s26tctkklxzr0000gp/T/ipykernel_58304/242195146.py:5: RuntimeWarning:

divide by zero encountered in double_scalars



In [62]:
fig = px.box(
    energy_ratio_df,
    x="payload_size",
    y="ratio", 
    color="unchanged_counter",
    facet_col="source_id"
)
fig

# What is BTP's overhead for building the tree

In [10]:
btp_energy_df = energy_df[energy_df["flood"] == False]

fig = px.bar(
    btp_energy_df,
    x="payload_size",
    y="mWb_rel",
    color="frame_type",
    facet_col="unchanged_counter",
    facet_row="source_id"
)
fig

# Strain on Individual Nodes

In [11]:
def sort_groups(x):
    sorted_x = x.sort_values(by="amount", ascending=False)
    sorted_x['sorted'] = range(len(sorted_x))
    
    return sorted_x

# strain_df = df.drop(df[df["flood"] == True].index)

strain_df = df.groupby(EXPERIMENT_IDENTIFIER + ["node_id"])["mWb"].sum().reset_index(name="amount")

strain_df = strain_df.groupby(EXPERIMENT_IDENTIFIER).agg(sort_groups).reset_index(drop=True)

strain_df["normalized"] = (strain_df["amount"] - strain_df["amount"].min()) / (strain_df["amount"].max() - strain_df["amount"].min())

/var/folders/26/2pp7vp553ms3s26tctkklxzr0000gp/T/ipykernel_92047/1837664311.py:11: FutureWarning:

Dropping invalid columns in DataFrameGroupBy.agg is deprecated. In a future version, a TypeError will be raised. Before calling .agg, select only columns which should be valid for the function.



In [13]:
fig = px.line(
    strain_df,
    x="sorted",
    y="normalized",
    color="flood",
    facet_col="unchanged_counter",
    facet_row="source_id",
    line_dash="payload_size"
)
fig